In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [25]:
dataset = pd.read_csv('ML_validating_v01_20220303.csv')
dataset.tail()

,station,EVI,Evapo,LST_Daily,LST_Diff,NDVI,TI,T_air,API,Clay,...,ESA-CCI,lat,lon,network,OMC,Porosity,Sand,Silt,Soil Moisture,Preci
199881,RendLake(Ina),0.240733,-0.004534,24.88,9.90,0.387445,15.238001,24.888287,8.485890,24.700001,...,0.224914,38.130001,-88.920000,ICN,5.03408,0.475472,5.900000,69.300003,0.1446,0.000435
199882,RendLake(Ina),0.227114,-0.003647,18.90,14.94,0.375854,15.238001,20.006017,30.926773,24.700001,...,0.234515,38.130001,-88.920000,ICN,5.03408,0.475472,5.900000,69.300003,0.1576,0.000858
199883,RendLake(Ina),0.223807,-0.001522,12.57,8.64,0.392707,15.238001,13.731508,35.938500,24.700001,...,0.332738,38.130001,-88.920000,ICN,5.03408,0.475472,5.900000,69.300003,0.3539,0.000000
199884,RendLake(Ina),0.465335,-0.005154,25.11,7.40,0.626214,15.238001,26.364821,45.631101,24.700001,...,0.267071,38.130001,-88.920000,ICN,5.03408,0.475472,5.900000,69.300003,0.3729,0.001716
199885,Topeka,0.726137,-0.005380,22.43,14.92,0.913425,18.246663,22.721974,23.579384,16.900000,...,0.321368,40.300000,-89.900003,ICN,5.63748,0.445283,61.799999,21.299999,0.2081,0.027052


In [26]:
dataset.columns

Index(['station', 'EVI', 'Evapo', 'LST_Daily', 'LST_Diff', 'NDVI', 'TI',
       'T_air', 'API', 'Clay', 'Date', 'Elevation', 'ESA-CCI', 'lat', 'lon',
       'network', 'OMC', 'Porosity', 'Sand', 'Silt', 'Soil Moisture', 'Preci'],
      dtype='object')

In [27]:
#remove string predictor: "network","station"
dataset = dataset.drop('network', axis=1)
dataset = dataset.drop('station', axis=1)
dataset = dataset.drop('ESA-CCI', axis=1)

In [28]:
dataset.columns

Index(['EVI', 'Evapo', 'LST_Daily', 'LST_Diff', 'NDVI', 'TI', 'T_air', 'API',
       'Clay', 'Date', 'Elevation', 'lat', 'lon', 'OMC', 'Porosity', 'Sand',
       'Silt', 'Soil Moisture', 'Preci'],
      dtype='object')

In [29]:
#deal with the date 
dataset2=pd.to_datetime(dataset['Date']) #move the datetime column to a new DF, and convert it to a recognized panda type 
DOY=dataset2.dt.dayofyear #create a Series "day", where the day of the year is stored
#also allows for weekofyear
Year= dataset2.dt.year #create a "year series"
dataset.insert(0, 'DOY', DOY) #insert the day series to the beggining of the dataset
dataset.insert(0, 'Year', Year)#insert the year series to the beggining of the dataset
dataset.drop(labels=['Date'], axis=1,inplace = True) #eliminate the date column

In [30]:
dataset.head()

,Year,DOY,EVI,Evapo,LST_Daily,LST_Diff,NDVI,TI,T_air,API,Clay,Elevation,lat,lon,OMC,Porosity,Sand,Silt,Soil Moisture,Preci
0,2016,178,0.267585,-0.002658,13.80,5.34,0.503435,9.724592,10.782256,42.469012,9.4,468.604279,68.330194,27.550618,37.6694,0.762264,55.5,35.099998,0.175417,0.121573
1,2016,179,0.264823,-0.003970,22.36,13.26,0.497070,9.724592,15.129432,38.514693,9.4,468.604279,68.330194,27.550618,37.6694,0.762264,55.5,35.099998,0.172500,0.001892
2,2016,182,0.256537,-0.003107,18.15,11.12,0.477974,9.724592,16.073048,41.956012,9.4,468.604279,68.330194,27.550618,37.6694,0.762264,55.5,35.099998,0.179500,6.832266
3,2016,197,0.220529,-0.003018,10.73,-11.00,0.403011,9.724592,14.346681,41.557064,9.4,468.604279,68.330194,27.550618,37.6694,0.762264,55.5,35.099998,0.179375,0.021294
4,2016,202,0.213499,-0.003174,14.76,5.98,0.396828,9.724592,14.414658,42.532913,9.4,468.604279,68.330194,27.550618,37.6694,0.762264,55.5,35.099998,0.176958,0.166529


In [31]:
Year.value_counts() #registries groped by years in our dataset. Why 2017 and 2018 have so few entries?

2017    61837
2016    58369
2015    25377
2018    18364
2014     8359
2010     6131
2013     6032
2012     4208
2009     4097
2011     3373
2008     2020
2007      697
2006      442
2005      394
2004       72
2002       68
2003       34
2001       11
2000        1
Name: Date, dtype: int64

In [34]:
s = dataset.pop('Soil Moisture')
dataset = pd.concat([dataset, s], 1) #move the dependent variable (soilMoisture to the end of the dataframe)
dataset

/tmp/ipykernel_4003/3782904172.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  dataset = pd.concat([dataset, s], 1) #move the dependent variable (soilMoisture to the end of the dataframe)


,Year,DOY,EVI,Evapo,LST_Daily,LST_Diff,NDVI,TI,T_air,API,Clay,Elevation,lat,lon,OMC,Porosity,Sand,Silt,Preci,Soil Moisture
0,2016,178,0.267585,-0.002658,13.80,5.34,0.503435,9.724592,10.782256,42.469012,9.400000,468.604279,68.330194,27.550618,37.66940,0.762264,55.500000,35.099998,0.121573,0.175417
1,2016,179,0.264823,-0.003970,22.36,13.26,0.497070,9.724592,15.129432,38.514693,9.400000,468.604279,68.330194,27.550618,37.66940,0.762264,55.500000,35.099998,0.001892,0.172500
2,2016,182,0.256537,-0.003107,18.15,11.12,0.477974,9.724592,16.073048,41.956012,9.400000,468.604279,68.330194,27.550618,37.66940,0.762264,55.500000,35.099998,6.832266,0.179500
3,2016,197,0.220529,-0.003018,10.73,-11.00,0.403011,9.724592,14.346681,41.557064,9.400000,468.604279,68.330194,27.550618,37.66940,0.762264,55.500000,35.099998,0.021294,0.179375
4,2016,202,0.213499,-0.003174,14.76,5.98,0.396828,9.724592,14.414658,42.532913,9.400000,468.604279,68.330194,27.550618,37.66940,0.762264,55.500000,35.099998,0.166529,0.176958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199881,2007,243,0.240733,-0.004534,24.88,9.90,0.387445,15.238001,24.888287,8.485890,24.700001,129.928635,38.130001,-88.920000,5.03408,0.475472,5.900000,69.300003,0.000435,0.144600
199882,2007,256,0.227114,-0.003647,18.90,14.94,0.375854,15.238001,20.006017,30.926773,24.700001,129.928635,38.130001,-88.920000,5.03408,0.475472,5.900000,69.300003,0.000858,0.157600
199883,2007,304,0.223807,-0.001522,12.57,8.64,0.392707,15.238001,13.731508,35.938500,24.700001,129.928635,38.130001,-88.920000,5.03408,0.475472,5.900000,69.300003,0.000000,0.353900
199884,2008,197,0.465335,-0.005154,25.11,7.40,0.626214,15.238001,26.364821,45.631101,24.700001,129.928635,38.130001,-88.920000,5.03408,0.475472,5.900000,69.300003,0.001716,0.372900


In [38]:
 #save the new dataset, used for training all the models
dataset.to_csv(path_or_buf=r'ML_validating_v01cleaned_20220329.csv', sep=',', index=False)